In [1]:
import findspark
findspark.init()
findspark.find()

'C:\\work\\spark-2.4.8-bin-hadoop2.7'

In [2]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("minTemp").getOrCreate()

In [3]:
text=spark.sparkContext.textFile("1800.csv")
text.take(5)

['ITE00100554,18000101,TMAX,-75,,,E,',
 'ITE00100554,18000101,TMIN,-148,,,E,',
 'GM000010962,18000101,PRCP,0,,,E,',
 'EZE00100082,18000101,TMAX,-86,,,E,',
 'EZE00100082,18000101,TMIN,-135,,,E,']

In [4]:
def lines_split(line):
    field=line.split(',')
    temp=float(field[3])*(9.0/5.0)+32.0
    return (field[0],field[2],temp)

In [5]:
data=text.map(lines_split)
data.take(5)

[('ITE00100554', 'TMAX', -103.0),
 ('ITE00100554', 'TMIN', -234.40000000000003),
 ('GM000010962', 'PRCP', 32.0),
 ('EZE00100082', 'TMAX', -122.80000000000001),
 ('EZE00100082', 'TMIN', -211.0)]

In [7]:
data=data.filter(lambda x: 'TMIN' in x[1]).map(lambda x: (x[0],x[2]))
data.take(5)

[('ITE00100554', -234.40000000000003),
 ('EZE00100082', -211.0),
 ('ITE00100554', -193.0),
 ('EZE00100082', -202.0),
 ('ITE00100554', -50.8)]

In [8]:
data=data.reduceByKey(lambda x,y: min(x,y))
data.take(5)

[('ITE00100554', -234.40000000000003), ('EZE00100082', -211.0)]